# オペレーティングシステム 演習 4

**このセルを編集して, 名前と学生証番号を書け.**

 * 名前 Name : 03-190503
 * 学生証番号 Student ID : 西山　晃人

**書けたら Shift + Enter で実行(入力を確定)させ, 保存(`Ctrl-S`)せよ**

* このページは, 以前の課題 オペレーティングシステム 演習 2 「同期, スレッドセーフなデータ」のうち, 課題2-2 スレッドセーフな有限バッファ(2) の不備を訂正したものです. 

* 混乱を防ぐため形式的に新しい課題として出題しています. 

* 問題の趣旨, 目標は課題2-2と全く同じですが, こちらから提供したベースとなるコード(動作をするためのコードでもある)が間違っていましたので, ここに正しいものを示します.

* 以前の課題 2-2 は回答不要です.

## <font color="blue">課題 4-1:</font> スレッドセーフな有限バッファ(2)

* _課題2-2を拡張して, queueが満杯の状態でenqが呼ばれたり, queueが空の状態でdeqが呼ばれたら, それぞれの動作が可能になるまで待つ動作にせよ_
* _ヒント: mutexに加えて条件変数を使う_

In [1]:
//%file: bb_4.h
//%cmd:

// answer-box
// ------- 解答をここに書く. write your answer here -------

#include <pthread.h>

/* 有限バッファ

             p->               q->
        +-+-+-+-+-+-+-+-+-+-+-+-+-+-+
   vals | | |x|x|x|x| ... |x|x| | | |
        +-+-+-+-+-+-+-+-+-+-+-+-+-+-+ 
         0 1 2 ...                     */

typedef struct {
  pthread_mutex_t m[1];
  pthread_cond_t c[1];
  long p;                       /* 次に要素が取られる場所 */
  long q;                       /* 次の要素が入る場所 */
  long C;                       /* 容量 */
  long * vals;                  /* C要素の配列 */
} bb_t;

/* 容量Cの有限バッファを作る */
bb_t * make_bb(long C);
/* bbの初期化*/
void bb_init(bb_t * bb);
/* xを挿入 */
long bb_enq(bb_t * bb, long x);
/* 取り出し */
long bb_deq(bb_t * bb);


In [2]:
//% file: bb_4.c
//% cmd: gcc -O3 -Wall -Wextra -fopenmp -o bb_4 bb_4.c

/* ------- このセルを修正して解答を書け. write your answer here ------- */

#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <pthread.h>
#include "bb_4.h"

/* チェック・エラー表示のための補助関数 */
int check_(int cond, const char * cond_s,
           char * file, int line, const char * fun) {
  if (!cond) {
    fprintf(stderr, "%s:%d:%s: check '%s' failed\n",
            file, line, fun, cond_s);
    abort();
  }
  return 1;                     /* OK */
}

#define check(x) check_((x) != 0, #x, __FILE__, __LINE__, __func__)

/* 容量Cの有限バッファを作る */
bb_t * make_bb(long C) {
  long * vals = calloc(C, sizeof(long));
  check(vals);
  bb_t * bb = malloc(sizeof(bb_t));
  check(bb);
  bb->p = 0;
  bb->q = 0;
  bb->C = C;
  bb->vals = vals;
  return bb;
}

void bb_init(bb_t * bb){
    pthread_mutex_init(bb->m, NULL);
    pthread_cond_init(bb->c, NULL);
}

/* 要素を入れる */
long bb_enq(bb_t * bb, long x) {
  pthread_mutex_lock(bb->m);
  while (bb->q - bb->p >= bb->C) {
    pthread_cond_wait(bb->c, bb->m);
  }
  long q = bb->q;
  bb->vals[q % bb->C] = x;
  bb->q = q + 1;
  pthread_cond_broadcast(bb->c);
  pthread_mutex_unlock(bb->m);
  return q;
}

/* 要素を取り出す */
long bb_deq(bb_t * bb) {
  pthread_mutex_lock(bb->m);
  while (bb->q - bb->p <= 0) {
    pthread_cond_wait(bb->c, bb->m);
  }
  long p = bb->p;
  long x = bb->vals[p % bb->C];
  bb->p = p + 1;
  pthread_cond_broadcast(bb->c);
  pthread_mutex_unlock(bb->m);
  return x;
}

/* スレッドを nthreads 個作って0 .. n-1 を1つずつbbにput */
long enqs(bb_t * bb, long n, long nthreads) {
#pragma omp parallel num_threads(nthreads)
#pragma omp for
  for (long i = 0; i < n; i++) {
    bb_enq(bb, i);
  }
  return 1;                     /* OK */
}

/* スレッドを nthreads 個作って合計n個の要素をbbからget.
   0 .. n - 1 が一回ずつ取り出されることをチェック. */
long deqs(bb_t * bb, long n, long nthreads) {
  /* check用配列. x が取り出されたところでcheck[x] = 1とする */
  char * a = calloc(1, n);
  for (long i = 0; i < n; i++) {
    a[i] = 0;
  }
#pragma omp parallel num_threads(nthreads)
#pragma omp for
  for (long i = 0; i < n; i++) {
    long x = bb_deq(bb);
    check(x >= 0);
    check(x < n);
    check(a[x] == 0);
    a[x] = 1;
  }
  for (long i = 0; i < n; i++) {
    check(a[i]);
  }
  free(a);
  return 1;
}

/* n個並行にenq; n個並行にdeq
   (enqとdeqは並行には行わない) */
long enqs_deqs(bb_t * bb, long n,
               long n_producers, long n_consumers) {
  enqs(bb, n, n_producers);
  return deqs(bb, n, n_consumers);
}

/* n個enqとn個deqを並行に */
long enqs_deqs_conc(bb_t * bb, long n,
                    long n_producers, long n_consumers) {
  int r = 0;
#pragma omp parallel num_threads(2)
  {
    int id = omp_get_thread_num();
    if (id == 0) {
      enqs(bb, n, n_producers);
    } else {
      assert(id == 1);
      r = deqs(bb, n, n_consumers);
    }
  }
  return r;
}

int main(int argc, char ** argv) {
  long C             = (argc > 1 ? atol(argv[1]) : 100000);
  long n             = (argc > 2 ? atol(argv[2]) : C * 3);
  long n_enq_threads = (argc > 3 ? atol(argv[3]) : 4); /* enq スレッドの数 */
  long n_deq_threads = (argc > 4 ? atol(argv[4]) : 8); /* deq スレッドの数 */
  long conc_enq_deq  = (argc > 5 ? atol(argv[5]) : 1); /* enq/deq を並行に行うか? */

  printf("queue capacity C = %ld\n", C);
  printf("items to enq/deq n = %ld\n", n);
  printf("enq threads = %ld\n", n_enq_threads);
  printf("deq threads = %ld\n", n_deq_threads);
  printf("concurrently enq/deq = %ld\n", conc_enq_deq);
  fflush(stdout);
  
  if (conc_enq_deq == 0 && n > C) {
    fprintf(stderr,
            "n items (%ld) > capacity (%ld);"
            " you must set conc_prod_cons = 0\n",
            n, C);
    exit(1);
  }
  omp_set_nested(1);

  bb_t * bb = make_bb(C);
  bb_init(bb);
  int r = 0;
  if (conc_enq_deq == 0) {
    r = enqs_deqs(bb, n, n_enq_threads, n_deq_threads);
  } else {
    r = enqs_deqs_conc(bb, n, n_enq_threads, n_deq_threads);
  }
  if (r) {
    printf("OK\n");
    return 0;
  } else {
    printf("NG\n");
    return 1;
  }
}


「容量 < enq回数」という場合でも正しく動作することを確認せよ

```
./bb_3 容量 enq回数(=deq回数) enqスレッド数 deqスレッド数 1
```

In [3]:
//% cmd: ./bb_4 10000 10000000 1 1 1
/* ------- このセルを実行して結果を確かめよ Execute this cell and check the result ------- */

queue capacity C = 10000
items to enq/deq n = 10000000
enq threads = 1
deq threads = 1
concurrently enq/deq = 1
OK


In [4]:
//% cmd: ./bb_4 10000 10000000 10 1 1
/* ------- このセルを実行して結果を確かめよ Execute this cell and check the result ------- */

queue capacity C = 10000
items to enq/deq n = 10000000
enq threads = 10
deq threads = 1
concurrently enq/deq = 1
OK


In [5]:
//% cmd: ./bb_4 10000 10000000 1 10 1
/* ------- このセルを実行して結果を確かめよ Execute this cell and check the result ------- */

queue capacity C = 10000
items to enq/deq n = 10000000
enq threads = 1
deq threads = 10
concurrently enq/deq = 1
OK


In [6]:
//% cmd: ./bb_4 10000 10000000 10 20 1
/* ------- このセルを実行して結果を確かめよ Execute this cell and check the result ------- */

queue capacity C = 10000
items to enq/deq n = 10000000
enq threads = 10
deq threads = 20
concurrently enq/deq = 1
OK


In [7]:
//% cmd: ./bb_4 10000 10000000 20 10 1
/* ------- このセルを実行して結果を確かめよ Execute this cell and check the result ------- */

queue capacity C = 10000
items to enq/deq n = 10000000
enq threads = 20
deq threads = 10
concurrently enq/deq = 1
OK


以下のような極端なケース(容量1)も試してみよ

In [8]:
//% cmd: ./bb_4 1 1000000 20 20 1
/* ------- このセルを実行して結果を確かめよ Execute this cell and check the result ------- */

queue capacity C = 1
items to enq/deq n = 1000000
enq threads = 20
deq threads = 20
concurrently enq/deq = 1
OK
